In [1]:
# import packages
import matplotlib.pyplot as plt
import pandas as pd

"""
IMPORTANT:
For this program to work the researcher needs to put both csv files
cycle1.csv and cycle2.csv in a folder called Observations. Run
DownloadMongoDB.ipynb to get the files. You will have to manually
put these new files in the Observations folder which will be located
in the same folder as this file.
"""

"""
Headers for Cycle1 DF:
_id, Original_Object_ID, MDJ, Magnitude, Magnitude_Error

Cycle1 Size: 28878 Asteroids

Headers for Cycle2 DF:
_id, Object_ID, MDJ, Magnitude, Magnitude_Error, Sector

Cycle2 Size: 18712 Asteroids
"""

# import data from csv file into a pandas df
cycle1_df = pd.read_csv("Observations/cycle1.csv", low_memory=False)
cycle2_df = pd.read_csv("Observations/cycle2.csv", low_memory=False)

In [ ]:
# TEMPLATE CODE FOR PLOTTING SPECIFIC ASTEROID ON THEIR OBJECT ID

# group each asteroid by their object id
asteroids = cycle1_df.groupby("Original_Object_ID")

count = 0

# plot each asteroid
for object_id, asteroid in asteroids:

    plt.figure(figsize=(10, 5))
    plt.errorbar(asteroid["MJD"], asteroid["Magnitude"], yerr = asteroid["Magnitude_Error"], 
                 fmt = "o", ecolor = "red", color = "orange", elinewidth = 1, capsize = 2)
    plt.title(f"Magnitude vs MJD for Object ID {object_id}")
    plt.xlabel("MJD")
    plt.ylabel("Magnitude")
    plt.gca().invert_yaxis()
    plt.grid(True)
    plt.show()

    count += 1

    # display the first 10
    if count > 10:
        break
